To Be Done:

Add code to handle IO Exceptions due to bad image files

Change hardcoded paths to use one parameter variable

Add code to create file structure

# Kaggle : 2 Sigma Connect : Rental Listing Inquiries

Purpose: To predict the interest level of a listing using the listing 
images only

Author : Chandan Panda 

Version: 1.0

In [1]:
from tqdm import tqdm
import numpy as np
import csv
import os
from PIL import Image
import matplotlib.pyplot as plt
from glob import glob
from keras.models import Model
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from random import uniform
import bcolz
import time
from keras.layers import merge
from keras.layers.core import Lambda
from keras.models import Model
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
from keras.layers import Input, Dense
import math
import pandas as pd
from keras.utils import np_utils as u
import utils; reload(utils)
from utils import *
import gc

%matplotlib inline
%cd '/cp/home/ubuntu/intel'


/home/ubuntu/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using Theano backend.
WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: unknown error)


/cp/home/ubuntu/intel


# Create datasets

We randomly divide the dataset into 5 folds and also keep a small sample dataset for debugging. Each fold contains about 54000 images

The images are structured into folders named high, medium or low based on the interest level of the listing in which the image was uploaded

We make sure that a listing's images are not scattered across both different folds!

In [ ]:
#Create 5 fold datasets

def setupFiles(path,NumberOfFolds,folder):
    os.chdir(path)
    photos=glob("*.jpg")

    CREATE_FOLDS = True

    if CREATE_FOLDS==True:
        photos=glob("*.jpg")
        shuffled_photos = np.random.permutation(photos)
        sample_size = int(round(len(photos) * (1/NumberOfFolds),0))

        for i in range(int(NumberOfFolds)):
            if(os.path.exists('/cp/home/ubuntu/intel/folds/fold_'+str(i))==False):
                os.mkdir('/cp/home/ubuntu/intel/folds/fold_'+str(i))
            if(os.path.exists('/cp/home/ubuntu/intel/folds/fold_'+str(i)+'/' + folder)==False):
                os.mkdir('/cp/home/ubuntu/intel/folds/fold_'+str(i)+'/' + folder)
            for photo in shuffled_photos[range(i*sample_size,min(((i*sample_size)+sample_size),len(shuffled_photos) )) ]:
                os.rename(photo, '/cp/home/ubuntu/intel/folds/'+'fold_'+str(i)+'/' + folder + '/'+ photo)

In [ ]:
setupFiles('/cp/home/ubuntu/intel/all/Type_1',5.0,'Type_1')
setupFiles('/cp/home/ubuntu/intel/all/Type_2',5.0,'Type_2')
setupFiles('/cp/home/ubuntu/intel/all/Type_3',5.0,'Type_3')

# Sample a few images 

# Initialize Pre-trained Network 

We obtain a version of the VGG network (a convolutional neural network by the Visual Geometry and Graphics group) that is trained of the ImageNet database to predict 1000 classes

This version includes batch normalization layers that ensure that the weights not not become too large and improves the training time. In addition, it normalizes the image by subtracting the RGB mean values from the ImageNet database

In [2]:
from vgg16bn import Vgg16BN
model = vgg_ft_bn(3)
model.summary()

40960/35363 [==================================] - 0s 
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  0           zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
____________________________________

We will now split the VGG network into the convolutional and dense layers. This is so that we can reuse the convolutional layers weights as it is and train only the final three dense layers. This will save enormous amount of time while making little difference to our final performance

In [3]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
gen=image.ImageDataGenerator(width_shift_range = 0.2 , height_shift_range = 0.2 , shear_range = 0.2 , zoom_range = 0.2)
target_size=(224,224)
batch_size = 64
class_mode='categorical'
shuffle=True
dirname = 'train'
model.compile(optimizer=Adam(1e-3),loss='categorical_crossentropy', metrics=['accuracy'])
conv_layers,fc_layers = split_at(model, Convolution2D)
conv_model = Sequential(conv_layers)

# Generate Features

Preprocess all images in sample, training and validation sets by passing through the VGG network convolutional layers after a bit of data augmentation. These features (i.e. 512 X 14 X 14 per image) will then be used as inputs  to a dense layer network that finally outputs the probability of the image being low, medium or high interest_level

Because of the large number of images (~200k training set, ~70k in validation set) and RAM constraint of 64 GB, we will divide the features into chunks of 10k images and use the fit_generator and evaluate_generator to iterate through each file chunk as we train and validate the dense layer network.

Also, the images are randomly width shifted, height shifted, sheared and zoomed to introduce bias in order to reduce variance

Each epoch will take approximately an hour!

In [6]:
def generateFeatures(dirname,CHUNK_SIZE = 10000):
    GENERATE_FEATURES = True
    if GENERATE_FEATURES==True:
        batches = gen.flow_from_directory('/cp/home/ubuntu/intel/folds/'+dirname, 
                                                         target_size=target_size,class_mode=class_mode, 
                                          shuffle=True, batch_size=1)
        labels_all = onehot(batches.classes) 
        totalImages = batches.nb_sample
        listing_ids_all = np.ndarray([totalImages,1],dtype=np.int64)
        listing_ids_all = [int((f.split('.')[0].split('/')[1])) for f in batches.filenames]
        batchIndex = 0
        numImages = 0
        images = np.ndarray([CHUNK_SIZE,3, 224, 224],dtype=np.float32)
        labels = np.ndarray([CHUNK_SIZE,3],dtype=np.float64)
        listing_ids = np.ndarray([CHUNK_SIZE,1],dtype=np.int64)   
        for i in range(totalImages):
            if (numImages > (CHUNK_SIZE - 1)) or (i == (totalImages-1)):
                labels = labels_all[batchIndex*CHUNK_SIZE:batchIndex*CHUNK_SIZE + numImages]
                listing_ids = listing_ids_all[batchIndex*CHUNK_SIZE:batchIndex*CHUNK_SIZE + numImages]
                save_array('features/' + dirname + '_labels_%d.dat'%batchIndex,labels[0:numImages])               
                save_array('features/' + dirname + '_listing_ids_%d.dat'%batchIndex,listing_ids[0:numImages])                     
                conv_feat = conv_model.predict_generator(generator=gen.flow(images[0:numImages],batch_size=batch_size)
                                                         ,val_samples=numImages)        
                save_array('features/' + dirname + '_trainX_%d.dat'%batchIndex,conv_feat[0:numImages])  
                batchIndex += 1
                images = np.ndarray([CHUNK_SIZE,3, 224, 224],dtype=np.float32)
                labels = np.ndarray([CHUNK_SIZE,3],dtype=np.float64)
                numImages = 0
                del(conv_feat)
                del(labels)
                gc.collect()            
            x,y = batches.next()
            images[numImages] = x[0]
            numImages += 1

In [11]:
generateFeatures('fold_0',CHUNK_SIZE=20000)
generateFeatures('fold_1',CHUNK_SIZE=20000)
generateFeatures('fold_2',CHUNK_SIZE=20000)
generateFeatures('fold_3',CHUNK_SIZE=20000)
generateFeatures('fold_4',CHUNK_SIZE=20000)

Found 1736 images belonging to 3 classes.
Found 1738 images belonging to 3 classes.
Found 675 images belonging to 3 classes.


Bad Image Files 5892,2845,5893

Move augmented features to another folder

In [12]:
%cd '/cp/home/ubuntu/intel/features'
photos=glob("*fold*.dat")
for photo in photos:
    os.rename(photo, '/cp/home/ubuntu/intel/features-aug/'+photo)
%cd '/cp/home/ubuntu/intel'

/cp/home/ubuntu/intel/features
/cp/home/ubuntu/intel


Function to randomize fold feature chunks. This is currently hardcoded to handle exactly 1 filechunk per fold!

In [13]:
def randomizeFeatures(fold,folder):
    os.chdir(folder)
    Files = glob(fold+'_trainX*.dat')
    chunk0 = load_array(Files[0])
    chunk0size = chunk0.shape[0]
    labels_chunk0 = load_array(Files[0].replace('trainX','labels'))
    listing_ids_chunk0 = load_array(Files[0].replace('trainX','listing_ids'))
    features = chunk0
    del(chunk0)
    gc.collect()
    labels = labels_chunk0
    del(labels_chunk0)
    gc.collect()
    listing_ids = listing_ids_chunk0
    del(listing_ids_chunk0)
    gc.collect()

    seq = np.random.permutation(range(features.shape[0]))

    tempArrayFeatures = np.ndarray([1,512, 14, 14],dtype=np.float32)
    tempArrayListingIds = 0
    tempArrayLabels = np.ndarray([1,3],dtype=np.float32)

    for i in range(features.shape[0]):
        tempArrayFeatures[0] = features[seq[i]]
        features[seq[i]] = features[i]
        features[i] = tempArrayFeatures[0]
        tempArrayListingIds = listing_ids[seq[i]]
        listing_ids[seq[i]] = listing_ids[i]
        listing_ids[i] = tempArrayListingIds
        tempArrayLabels[0] = labels[seq[i]]
        labels[seq[i]] = labels[i]
        labels[i] = tempArrayLabels[0]

    save_array(Files[0],features[range(chunk0size)])
    save_array(Files[0].replace('trainX','labels'),labels[range(chunk0size)])
    save_array(Files[0].replace('trainX','listing_ids'),listing_ids[range(chunk0size)])
    os.chdir('/cp/home/ubuntu/intel')
    return

In [14]:
def checkLabels(folder):    
    os.chdir(folder)
    chunks=glob("fold_*_labels_*.dat")
    gc.collect()
    for chunk in chunks:
        labels = load_array(chunk)
        print(chunk,np.sum(labels[:,0]),np.sum(labels[:,1]),np.sum(labels[:,2]))
    os.chdir('/cp/home/ubuntu/intel')
    return

In [15]:
checkLabels('/cp/home/ubuntu/intel/features-aug')

('fold_0_labels_0.dat', 335.0, 938.0, 464.0)
('fold_2_labels_0.dat', 335.0, 936.0, 464.0)
('fold_3_labels_0.dat', 335.0, 938.0, 464.0)
('fold_1_labels_0.dat', 334.0, 938.0, 464.0)
('fold_4_labels_0.dat', 54.0, 156.0, 464.0)


In [16]:
randomizeFeatures('fold_0','/cp/home/ubuntu/intel/features-aug')
randomizeFeatures('fold_1','/cp/home/ubuntu/intel/features-aug')
randomizeFeatures('fold_2','/cp/home/ubuntu/intel/features-aug')
randomizeFeatures('fold_3','/cp/home/ubuntu/intel/features-aug')
randomizeFeatures('fold_4','/cp/home/ubuntu/intel/features-aug')

In [17]:
checkLabels('/cp/home/ubuntu/intel/features-aug')

('fold_0_labels_0.dat', 335.0, 938.0, 464.0)
('fold_2_labels_0.dat', 335.0, 936.0, 464.0)
('fold_3_labels_0.dat', 335.0, 938.0, 464.0)
('fold_1_labels_0.dat', 334.0, 938.0, 464.0)
('fold_4_labels_0.dat', 54.0, 156.0, 464.0)


Create a dense layer network with batch normalization to reduce the time it takes to train. Also include a drop out of 0.4 to improve the validation accuracy. This network takes the pre-processed features from the saved numpy arrays and outputs 3 probabilities - interest level will be low, medium or high

In [18]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        BatchNormalization(axis=1),
        Dropout(p/4),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(3, activation='softmax')
    ]

# Train the Network

Training a network on such a large dataset requires us to use the generator functions in keras. These functions loop 
indefinitely and return to the yield statement the next time the function is called. This allows us to iterate through the 
image features one batch at a time till all of them are passed through the network. Once a numpy array is completed, the next 
one is loaded from memory so that at any given time, a maximum of 20k images are in memory.

The metrics such as validation categorical cross-entropy loss and accuracy are logged to a log file so that even if 
we lose the connection midway, we can review how the metrics changed across the epochs

Finally, we keep saving the best model's weights

In [19]:
def trainMyNw(nb_epoch,fold,folder):
    
    myGen=image.ImageDataGenerator()
    best_loss_metric = 999

    for e in tqdm(range(nb_epoch)):
        
        start = time.clock()
        f = open('/cp/home/ubuntu/intel/results/log.txt', 'a')       
        featureFiles = glob(folder + '/' + 'fold_*_trainX_*.dat')        
        chunkCount=0
        
        for files in featureFiles:
            if files.find(fold) == -1:
                features = load_array(files)
                labels = load_array(files.replace('trainX','labels'))
                chunkCount += 1
                numImages = features.shape[0]
                bn_model.fit_generator(generator=myGen.flow(features,labels),samples_per_epoch = numImages,nb_epoch=1,verbose=0)  
                del(features)
                del(labels)
                gc.collect()
            
        featureFiles = glob(folder + '/' + fold + '_trainX_*.dat')        
        accuracy = 0
        loss = 0
        avg_accuracy = 0
        avg_loss = 0
        chunkCount = 0
        totalImages = 0

        for files in featureFiles:
            features = load_array(files)
            labels = load_array(files.replace('trainX','labels'))
            chunkCount += 1
            numImages = features.shape[0]
            loss_metrics = bn_model.evaluate_generator(generator=myGen.flow(features,labels),
                                                       val_samples = numImages) 
            accuracy += (loss_metrics[1]*numImages)
            loss += (loss_metrics[0]*numImages)
            totalImages += numImages
            del(features)
            del(labels) 
            gc.collect()            
            
        if chunkCount>0: 
            avg_accuracy = accuracy/totalImages
            avg_loss = loss/totalImages
            
        print bn_model.metrics_names,avg_loss,avg_accuracy, "time :", time.clock() - start
        f.write(str(bn_model.metrics_names))
        f.write(" : ")
        f.write(str(avg_loss))
        f.write(" : ")
        f.write(str(avg_accuracy))
        f.write(" : time :")
        f.write(str(time.clock() - start))
        f.write("\n")        
        f.close() 
        if (avg_loss<best_loss_metric):
            bn_model.save_weights('/cp/home/ubuntu/intel/results/best.hdf5') 
            best_loss_metric = avg_loss
            
    return 

We train the network with a high learning rate once and then gradually decrease the learning rate as we go through about 12 epochs

A dropout of 0.2 is optimal for training on train dataset while a dropout of 0.4 suits the valid dataset. Train the network twice, once on the training and once on the validation dataset and save the versions with the lowest loss for later use for stacking

Save best model weights

In [ ]:
p=0.9
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
trainMyNw(1,'fold_0','features-aug')
bn_model.optimizer.lr=0.001
trainMyNw(1,'fold_0','features-aug')
os.rename('/cp/home/ubuntu/intel/results/best.hdf5', '/cp/home/ubuntu/data/results/fold_0_XXXXn.hdf5')

In [ ]:
p=0.9
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
trainMyNw(1,'fold_1','features-aug')
bn_model.optimizer.lr=0.001
trainMyNw(1,'fold_1','features-aug')
os.rename('/cp/home/ubuntu/intel/results/best.hdf5', '/cp/home/ubuntu/data/results/fold_1_XXXXn.hdf5')

In [ ]:
p=0.9
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
trainMyNw(1,'fold_2','features-aug')
bn_model.optimizer.lr=0.001
trainMyNw(1,'fold_2','features-aug')
os.rename('/cp/home/ubuntu/intel/results/best.hdf5', '/cp/home/ubuntu/data/results/fold_2_XXXXn.hdf5')

In [ ]:
p=0.9
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
trainMyNw(1,'fold_3','features-aug')
bn_model.optimizer.lr=0.001
trainMyNw(1,'fold_3','features-aug')
os.rename('/cp/home/ubuntu/intel/results/best.hdf5', '/cp/home/ubuntu/data/results/fold_3_XXXXn.hdf5')

In [ ]:
p=0.9
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
trainMyNw(1,'fold_4','features-aug')
bn_model.optimizer.lr=0.001
trainMyNw(1,'fold_4','features-aug')
os.rename('/cp/home/ubuntu/intel/results/best.hdf5', '/cp/home/ubuntu/data/results/fold_4_XXXXn.hdf5')

Train final model on entire dataset. Manually set the number of epochs based on analysis of training logs

In [ ]:
p=0.9
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
trainMyNw(1,'sample','features-aug')
bn_model.optimizer.lr=0.001
trainMyNw(1,'sample','features-aug')

Save model weights

In [ ]:
bn_model.save_weights('/cp/home/ubuntu/data/results/fold_all_XXXXn.hdf5') 

Create a folder with photos for entire dataset

# Generate features for final scoring

Generate features for Test Dataset

In [ ]:
gen=image.ImageDataGenerator()
generateFeatures('test',CHUNK_SIZE=20000)

Generate features for all folds without data augmentation

In [ ]:
gen=image.ImageDataGenerator()
generateFeatures('fold_0',CHUNK_SIZE=20000)
generateFeatures('fold_1',CHUNK_SIZE=20000)
generateFeatures('fold_2',CHUNK_SIZE=20000)
generateFeatures('fold_3',CHUNK_SIZE=20000)
generateFeatures('fold_4',CHUNK_SIZE=20000)

In [ ]:
randomizeFeatures('fold_0','/cp/home/ubuntu/intel/features')
randomizeFeatures('fold_1','/cp/home/ubuntu/intel/features')
randomizeFeatures('fold_2','/cp/home/ubuntu/intel/features')
randomizeFeatures('fold_3','/cp/home/ubuntu/intel/features')
randomizeFeatures('fold_4','/cp/home/ubuntu/intel/features')

In [ ]:
checkLabels('/cp/home/ubuntu/intel/features')

# Score Datasets

In [ ]:
def scoreMyData(fold,weights_path):
    gen=image.ImageDataGenerator()
    bn_model.load_weights(weights_path) 
    featureFiles = glob('features/' + fold + '_trainX_*.dat')        

    for files in featureFiles:
        features = load_array(files)
        numImages = features.shape[0]
        scores = np.ndarray([numImages,3],dtype=np.float64)
        scores = bn_model.predict_generator(generator=gen.flow(features),val_samples = numImages)       
        save_array('predictions/' + files.split('/')[1].replace('trainX','scores'),scores)               
        del(scores)
        del(features)
        gc.collect()            
    return 

In [ ]:
scoreMyData('sample','/cp/home/ubuntu/intel/results/fold_0_XXXXn.hdf5')

Score all folds

In [ ]:
scoreMyData('fold_0','/cp/home/ubuntu/intel/results/fold_0_XXXXn.hdf5')
scoreMyData('fold_1','/cp/home/ubuntu/intel/results/fold_1_XXXXn.hdf5')
scoreMyData('fold_2','/cp/home/ubuntu/intel/results/fold_2_XXXXn.hdf5')
scoreMyData('fold_3','/cp/home/ubuntu/intel/results/fold_3_XXXXn.hdf5')
scoreMyData('fold_4','/cp/home/ubuntu/intel/results/fold_4_XXXXn.hdf5')

Score test dataset

In [ ]:
scoreMyData('test','/cp/home/ubuntu/intel/results/fold_all_XXXXn.hdf5')

# Create final Predictions file extract

In [ ]:
%cd '/cp/home/ubuntu/intel/predictions'
scoreFiles = glob('*scores*.dat') 
listing_ids_all = load_array('/cp/home/ubuntu/intel/features/' + scoreFiles[0].replace('scores','listing_ids'))
scores_all = load_array(scoreFiles[0])

for i in range(2,len(scoreFiles)):
    files =  scoreFiles[i]
    listing_ids = load_array('/cp/home/ubuntu/intel/features/' + files.replace('scores','listing_ids'))
    scores = load_array(files)
    listing_ids_all = np.concatenate((listing_ids_all, listing_ids))
    scores_all = np.vstack((scores_all, scores))

listing_ids_all.shape,scores_all.shape  

import pandas as pd 

df = pd.DataFrame(listing_ids_all)
df.to_csv("listing_ids_all.csv")

df = pd.DataFrame(scores_all)
df.to_csv("scores_all.csv")


# That's all Folks !

# Scrapbook

In [27]:
features1 = load_array('/cp/home/ubuntu/intel/features-aug/fold_2_trainX_0.dat')
labels1 = load_array('/cp/home/ubuntu/intel/features-aug/fold_2_labels_0.dat')
features2 = load_array('/cp/home/ubuntu/intel/features-aug/fold_3_trainX_0.dat')
labels2 = load_array('/cp/home/ubuntu/intel/features-aug/fold_3_labels_0.dat')
features3 = load_array('/cp/home/ubuntu/intel/features-aug/fold_1_trainX_0.dat')
labels3 = load_array('/cp/home/ubuntu/intel/features-aug/fold_1_labels_0.dat')

features = np.vstack((features1,features2,features3))
labels = np.vstack((labels1,labels2,labels3))

del(features1)
del(features2)
del(labels1)
del(labels2)
gc.collect()

388

In [23]:
features1 = load_array('/cp/home/ubuntu/intel/features-aug/fold_0_trainX_0.dat')
labels1 = load_array('/cp/home/ubuntu/intel/features-aug/fold_0_labels_0.dat')
features2 = load_array('/cp/home/ubuntu/intel/features-aug/fold_4_trainX_0.dat')
labels2 = load_array('/cp/home/ubuntu/intel/features-aug/fold_4_labels_0.dat')

features_val = np.vstack((features1,features2))
labels_val = np.vstack((labels1,labels2))

del(features1)
del(features2)
del(labels1)
del(labels2)
gc.collect()

0

In [28]:
features.shape,features_val.shape,labels.shape,labels_val.shape

((5208, 512, 14, 14), (2411, 512, 14, 14), (5208, 3), (2411, 3))

In [53]:
p=0.8
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [54]:
bn_model.fit(features, labels, batch_size=64, shuffle=True ,nb_epoch=2,validation_data=(features_val, labels_val),verbose=1)
bn_model.optimizer.lr = 0.0001
bn_model.fit(features, labels, batch_size=64, shuffle=True ,nb_epoch=2,validation_data=(features_val, labels_val),verbose=1)
bn_model.optimizer.lr = 0.00001
bn_model.fit(features, labels, batch_size=64, shuffle=True ,nb_epoch=5,validation_data=(features_val, labels_val),verbose=1)

Train on 5208 samples, validate on 2411 samples
Epoch 1/2
5208/5208 [==============================] - 3s - loss: 1.5765 - acc: 0.4180 - val_loss: 2.3474 - val_acc: 0.3625
Epoch 2/2
5208/5208 [==============================] - 3s - loss: 1.2145 - acc: 0.4618 - val_loss: 1.4567 - val_acc: 0.4467
Train on 5208 samples, validate on 2411 samples
Epoch 1/2
5208/5208 [==============================] - 3s - loss: 1.1378 - acc: 0.4900 - val_loss: 1.2532 - val_acc: 0.4500
Epoch 2/2
5208/5208 [==============================] - 3s - loss: 1.0638 - acc: 0.5173 - val_loss: 1.1391 - val_acc: 0.4538
Train on 5208 samples, validate on 2411 samples
Epoch 1/5
5208/5208 [==============================] - 3s - loss: 0.9942 - acc: 0.5503 - val_loss: 1.2072 - val_acc: 0.4305
Epoch 2/5
5208/5208 [==============================] - 3s - loss: 0.9006 - acc: 0.6052 - val_loss: 1.1761 - val_acc: 0.4430
Epoch 3/5
5208/5208 [==============================] - 3s - loss: 0.8028 - acc: 0.6455 - val_loss: 1.2069 - val_

In [ ]:
p=0.5
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=5,validation_data=(features_val, labels_val),verbose=1)

In [ ]:
p=0.5
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=10,validation_data=(features_val, labels_val),verbose=1)

In [ ]:
p=0.5
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=5,validation_data=(features_val, labels_val),verbose=1)

In [ ]:
p=0.5
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.005), loss='categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=5,validation_data=(features_val, labels_val),verbose=1)

In [ ]:
p=0.5
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=5,validation_data=(features_val, labels_val),verbose=1)

In [ ]:
p=0.6
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=5,validation_data=(features_val, labels_val),verbose=1)

In [ ]:
p=0.7
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=5,validation_data=(features_val, labels_val),verbose=1)

In [ ]:
p=0.7
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=1,validation_data=(features_val, labels_val),verbose=1)
bn_model.optimizer.lr = 0.001
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=1,validation_data=(features_val, labels_val),verbose=1)
bn_model.optimizer.lr = 0.00001
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=3,validation_data=(features_val, labels_val),verbose=1)

In [ ]:
p=0.7
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=1,validation_data=(features_val, labels_val),verbose=1)
bn_model.optimizer.lr = 0.00001
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=3,validation_data=(features_val, labels_val),verbose=1)

In [ ]:
p=0.1
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=5,validation_data=(features_val, labels_val),verbose=1)
bn_model.optimizer.lr = 0.001
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=1,validation_data=(features_val, labels_val),verbose=1)

In [ ]:
p=0.9
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=1,validation_data=(features_val, labels_val),verbose=1)
bn_model.optimizer.lr=0.001
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=2,validation_data=(features_val, labels_val),verbose=1)
bn_model.optimizer.lr=0.0001
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=3,validation_data=(features_val, labels_val),verbose=1)
bn_model.optimizer.lr=0.00001
bn_model.fit(features, labels, batch_size=32, shuffle=True ,nb_epoch=3,validation_data=(features_val, labels_val),verbose=1)

In [ ]:
gc.collect()

In [ ]:
import inspect
print inspect.getsource(bn_model.evaluate_generator)